In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np

2024-01-25 11:13:53.406347: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("../Data/2.sample_dataset.csv")
df = df.dropna(subset=['words'])
df

,target,id,date,flag,user,text,words
0,0,2054665466,Sat Jun 06 07:50:24 PDT 2009,NO_QUERY,boring_alice,ive had awesome day but the sun is missing wan...,ive awesome day sun miss want well weather
1,1,1823190427,Sat May 16 21:20:00 PDT 2009,NO_QUERY,yassychan,you will do great saw kevin teaching you,great saw kevin teach
2,0,1826975071,Sun May 17 09:43:20 PDT 2009,NO_QUERY,PRNCSmuriel3,its cold in md too,cold md
3,0,2202406793,Tue Jun 16 21:44:37 PDT 2009,NO_QUERY,mariazimmerman,does anyone know the girl that died of swine f...,anyone know girl die swine flu maybe go ucsd f...
4,0,2242106714,Fri Jun 19 11:46:38 PDT 2009,NO_QUERY,katrinachelsea,watching amelie and wishing was french,watch amelie wish french
...,...,...,...,...,...,...,...
999995,1,1679383266,Sat May 02 09:13:22 PDT 2009,NO_QUERY,Shawna1976,goodmorning jordan needs to talk to you,goodmorning jordan need talk
999996,1,2053651702,Sat Jun 06 05:16:26 PDT 2009,NO_QUERY,harlequinxgirl,good mprninh to you too,good mprninh
999997,0,1563449159,Sun Apr 19 22:52:58 PDT 2009,NO_QUERY,Cynthi_ocho,cant believe spring break is coming to an end,cant believe spring break come end
999998,0,1751244042,Sat May 09 19:00:02 PDT 2009,NO_QUERY,Ericanderson09,everyone have texted in the last hour complete...,everyone texted last hour completely ignore im...


In [4]:
X = df['words']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(796392,)
(796392,)
(199099,)
(199099,)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np

# Charger votre DataFrame
# df = ...

# Divisez le DataFrame en ensembles d'entraînement et de test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenisation des textes avec des mots
tokenized_texts = [text.split() for text in train_data['words']]

# Entraînement du modèle Word2Vec
embedding_dim = 100
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Tokenisation des textes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['words'])

# Convertir les textes en séquences d'entiers
train_sequences = tokenizer.texts_to_sequences(train_data['words'])
test_sequences = tokenizer.texts_to_sequences(test_data['words'])

# Rembourrage des séquences pour avoir des tailles égales
max_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_data_pad = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_data_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Construction de la matrice d'embedding à partir du modèle Word2Vec
embedding_matrix = []
for word, index in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix.append(word2vec_model.wv[word])
    else:
        embedding_matrix.append([0] * embedding_dim)

embedding_matrix = np.array(embedding_matrix)

# Création du modèle CNN avec la matrice d'embedding pré-entraînée
model_word2vec = Sequential()
model_word2vec.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model_word2vec.add(Conv1D(128, 5, activation='relu'))
model_word2vec.add(GlobalMaxPooling1D())
model_word2vec.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model_word2vec.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Définir les labels d'entraînement et de test
train_labels = train_data['sentiment'].values
test_labels = test_data['sentiment'].values

# Entraîner le modèle
model_word2vec.fit(train_data_pad, train_labels, epochs=5, validation_data=(test_data_pad, test_labels))

# Évaluation sur l'ensemble de test
loss, accuracy = model_word2vec.evaluate(test_data_pad, test_labels)
print(f"Test Accuracy: {accuracy*100:.2f}%")
